In [1]:
import pandas as pd
import geopandas as gpd
import os
pd.set_option('display.max_columns', None)

In [2]:
manzanas = gpd.read_file(os.path.join("..", "datos", "01_procesos", "manzanas_datoscenso_zecditvallejo.gpkg"))
manzanas.head(1)

,CVEGEO,pob_total,pob_fem,pob_masc,pob_15_24,pob_15_29,pob_15_64,pob_30_49,pob_50_59,pob_60_mas,pob_alfabeta,pob_analfabeta,pob_sin_escolaridad,pob_basica_comp,pob_posbasica,pob_media_sup,pob_superior,pea,pob_ocupada,pob_ocup_sup,pob_desocupada,pob_no_pea,viv_habitadas,viv_part_hab,ocup_viv,viv_deshabitadas,viv_elect,viv_agua,viv_drenaje,viv_serv_comp,viv_comp,viv_cel,viv_internet,viv_sin_comp_int,viv_1dorm,viv_hacinada,den_pob_m2,geometry
0,0900200010398005,142.0,71.0,71.0,26.0,39.0,105.0,33.0,25.0,22.0,119.0,0.0,0.0,21.0,84.0,38.0,34.0,79.0,76.0,28.0,3.0,48.0,42.0,42.0,142.0,14.0,42.0,42.0,42.0,42.0,26.0,39.0,32.0,9.0,15.0,12.0,NaN,"MULTIPOLYGON (((483432.725 2153439.42, 483450...."


# Cálculo de columnas de porcentaje

Se crean columnas `pct_*` para cada variable pertinente usando el denominador adecuado según el universo de referencia del Censo 2020:

| Grupo | Denominador |
|-------|-------------|
| Grupos de edad | `pob_total` |
| Educación (15+) | `pob_15_64` |
| Participación económica (`pea`, `pob_no_pea`) | `pob_15_64` |
| Ocupación / desocupación | `pea` |
| Ocupados con ed. superior | `pob_ocupada` |
| Infraestructura y tecnología de vivienda | `viv_part_hab` |
| Viviendas deshabitadas | `viv_habitadas + viv_deshabitadas` |

Los valores `NaN` en denominadores (manzanas con datos confidenciales -8) producen `NaN` en el porcentaje correspondiente, lo cual es el comportamiento correcto.

In [3]:
def agregar_porcentajes(gdf):
    """
    Agrega columnas pct_* al GeoDataFrame de manzanas o AGEBs del Censo 2020.
    Los NaN en denominadores (datos confidenciales) se propagan correctamente.
    Los denominadores == 0 (zonas sin población) producen NaN vía división por cero.
    """
    df = gdf.copy()

    def pct(num, den):
        """Porcentaje con manejo de división por cero → NaN."""
        return (num / den.replace(0, float('nan')) * 100).round(2)

    # ── GRUPOS DE EDAD (denominador: pob_total) ────────────────────────────
    for col in ['pob_fem', 'pob_masc', 'pob_15_24', 'pob_15_29',
                'pob_15_64', 'pob_30_49', 'pob_50_59', 'pob_60_mas']:
        df[f'pct_{col.removeprefix("pob_")}'] = pct(df[col], df['pob_total'])

    # ── EDUCACIÓN (denominador: pob_15_64 — mejor proxy de "pob 15+") ──────
    for col in ['pob_alfabeta', 'pob_analfabeta', 'pob_sin_escolaridad',
                'pob_basica_comp', 'pob_posbasica', 'pob_media_sup', 'pob_superior']:
        df[f'pct_{col.removeprefix("pob_")}'] = pct(df[col], df['pob_15_64'])

    # ── ACTIVIDAD ECONÓMICA ────────────────────────────────────────────────
    # Tasa de participación y de inactividad (base: pob_15_64)
    df['pct_pea']     = pct(df['pea'],         df['pob_15_64'])
    df['pct_no_pea']  = pct(df['pob_no_pea'],  df['pob_15_64'])

    # Tasa de ocupación y desocupación (base: pea)
    df['pct_ocupada']     = pct(df['pob_ocupada'],     df['pea'])
    df['pct_desocupada']  = pct(df['pob_desocupada'],  df['pea'])

    # Ocupados con educación superior (base: pob_ocupada)
    df['pct_ocup_sup'] = pct(df['pob_ocup_sup'], df['pob_ocupada'])

    # ── VIVIENDA (denominador: viv_part_hab) ──────────────────────────────
    for col in ['viv_elect', 'viv_agua', 'viv_drenaje', 'viv_serv_comp',
                'viv_comp', 'viv_cel', 'viv_internet', 'viv_sin_comp_int',
                'viv_1dorm', 'viv_hacinada']:
        df[f'pct_{col}'] = pct(df[col], df['viv_part_hab'])

    # Tasa de desocupación de viviendas (base: total viviendas = hab + deshabitadas)
    df['pct_viv_deshabitadas'] = pct(
        df['viv_deshabitadas'],
        df['viv_habitadas'] + df['viv_deshabitadas']
    )

    return df

In [4]:
manzanas.head(1)

,CVEGEO,pob_total,pob_fem,pob_masc,pob_15_24,pob_15_29,pob_15_64,pob_30_49,pob_50_59,pob_60_mas,pob_alfabeta,pob_analfabeta,pob_sin_escolaridad,pob_basica_comp,pob_posbasica,pob_media_sup,pob_superior,pea,pob_ocupada,pob_ocup_sup,pob_desocupada,pob_no_pea,viv_habitadas,viv_part_hab,ocup_viv,viv_deshabitadas,viv_elect,viv_agua,viv_drenaje,viv_serv_comp,viv_comp,viv_cel,viv_internet,viv_sin_comp_int,viv_1dorm,viv_hacinada,den_pob_m2,geometry
0,0900200010398005,142.0,71.0,71.0,26.0,39.0,105.0,33.0,25.0,22.0,119.0,0.0,0.0,21.0,84.0,38.0,34.0,79.0,76.0,28.0,3.0,48.0,42.0,42.0,142.0,14.0,42.0,42.0,42.0,42.0,26.0,39.0,32.0,9.0,15.0,12.0,NaN,"MULTIPOLYGON (((483432.725 2153439.42, 483450...."


In [5]:
# Aplicar a nivel manzana
manzanas_pct = agregar_porcentajes(manzanas)

# Verificar columnas generadas
cols_pct = [c for c in manzanas_pct.columns if c.startswith('pct_')]
print(f"Columnas pct_ generadas: {len(cols_pct)}")
print(cols_pct)

manzanas_pct[cols_pct].describe().T[['mean', 'min', 'max', 'count']].round(2)

Columnas pct_ generadas: 31
['pct_fem', 'pct_masc', 'pct_15_24', 'pct_15_29', 'pct_15_64', 'pct_30_49', 'pct_50_59', 'pct_60_mas', 'pct_alfabeta', 'pct_analfabeta', 'pct_sin_escolaridad', 'pct_basica_comp', 'pct_posbasica', 'pct_media_sup', 'pct_superior', 'pct_pea', 'pct_no_pea', 'pct_ocupada', 'pct_desocupada', 'pct_ocup_sup', 'pct_viv_elect', 'pct_viv_agua', 'pct_viv_drenaje', 'pct_viv_serv_comp', 'pct_viv_comp', 'pct_viv_cel', 'pct_viv_internet', 'pct_viv_sin_comp_int', 'pct_viv_1dorm', 'pct_viv_hacinada', 'pct_viv_deshabitadas']


,mean,min,max,count
pct_fem,51.21,0.0,100.00,5207.0
pct_masc,45.89,0.0,88.89,5207.0
pct_15_24,12.74,0.0,42.86,5207.0
pct_15_29,20.28,0.0,50.00,5207.0
pct_15_64,68.43,0.0,100.00,5207.0
pct_30_49,28.86,0.0,100.00,5207.0
pct_50_59,12.91,0.0,58.82,5207.0
pct_60_mas,19.49,0.0,83.33,5207.0
pct_alfabeta,119.75,0.0,261.38,5056.0
pct_analfabeta,0.50,0.0,50.00,5056.0


In [6]:
# Nivel AGEB
agebs = gpd.read_file(os.path.join("..", "datos", "01_procesos", "agebs_datoscenso_zecditvallejo.gpkg"))
agebs = agregar_porcentajes(agebs)

print(f"Manzanas: {manzanas.shape} | AGEBs: {agebs.shape}")
agebs[cols_pct].describe().T[['mean', 'min', 'max', 'count']].round(2)

Manzanas: (5700, 38) | AGEBs: (222, 68)


,mean,min,max,count
pct_fem,51.88,0.00,64.00,218.0
pct_masc,46.74,0.00,62.50,218.0
pct_15_24,13.37,0.00,29.17,218.0
pct_15_29,21.05,0.00,41.67,218.0
pct_15_64,69.59,0.00,87.50,218.0
pct_30_49,29.59,0.00,42.64,218.0
pct_50_59,13.28,0.00,17.98,218.0
pct_60_mas,19.62,0.00,34.39,218.0
pct_alfabeta,118.84,100.00,147.44,215.0
pct_analfabeta,1.21,0.00,17.71,215.0


In [9]:
manzanas_pct.head(1)

,CVEGEO,pob_total,pob_fem,pob_masc,pob_15_24,pob_15_29,pob_15_64,pob_30_49,pob_50_59,pob_60_mas,pob_alfabeta,pob_analfabeta,pob_sin_escolaridad,pob_basica_comp,pob_posbasica,pob_media_sup,pob_superior,pea,pob_ocupada,pob_ocup_sup,pob_desocupada,pob_no_pea,viv_habitadas,viv_part_hab,ocup_viv,viv_deshabitadas,viv_elect,viv_agua,viv_drenaje,viv_serv_comp,viv_comp,viv_cel,viv_internet,viv_sin_comp_int,viv_1dorm,viv_hacinada,den_pob_m2,geometry,pct_fem,pct_masc,pct_15_24,pct_15_29,pct_15_64,pct_30_49,pct_50_59,pct_60_mas,pct_alfabeta,pct_analfabeta,pct_sin_escolaridad,pct_basica_comp,pct_posbasica,pct_media_sup,pct_superior,pct_pea,pct_no_pea,pct_ocupada,pct_desocupada,pct_ocup_sup,pct_viv_elect,pct_viv_agua,pct_viv_drenaje,pct_viv_serv_comp,pct_viv_comp,pct_viv_cel,pct_viv_internet,pct_viv_sin_comp_int,pct_viv_1dorm,pct_viv_hacinada,pct_viv_deshabitadas
0,0900200010398005,142.0,71.0,71.0,26.0,39.0,105.0,33.0,25.0,22.0,119.0,0.0,0.0,21.0,84.0,38.0,34.0,79.0,76.0,28.0,3.0,48.0,42.0,42.0,142.0,14.0,42.0,42.0,42.0,42.0,26.0,39.0,32.0,9.0,15.0,12.0,NaN,"MULTIPOLYGON (((483432.725 2153439.42, 483450....",50.0,50.0,18.31,27.46,73.94,23.24,17.61,15.49,113.33,0.0,0.0,20.0,80.0,36.19,32.38,75.24,45.71,96.2,3.8,36.84,100.0,100.0,100.0,100.0,61.9,92.86,76.19,21.43,35.71,28.57,25.0


In [12]:
manzanas_pct['area_m2'] = manzanas_pct.geometry.area
manzanas_pct['den_pob_m2']= manzanas_pct['pob_total'] / manzanas_pct['area_m2']
manzanas_pct['den_pob_ha'] = manzanas_pct['pob_total'] / (manzanas_pct['area_m2'] / 10000)

# Exportar con porcentajes

In [13]:
ruta_finales = os.path.join("..", "datos", "02_finales")
os.makedirs(ruta_finales, exist_ok=True)

manzanas_pct.to_crs("EPSG:4326").to_file(os.path.join(ruta_finales, "manzanas_censo_pct_zecditvallejo.geojson"))
agebs.to_file(os.path.join(ruta_finales, "agebs_censo_pct_zecditvallejo.geojson"))

print("Exportados:")
print(f"  manzanas_censo_pct_zecditvallejo.geojson  → {manzanas.shape[0]:,} registros | {manzanas.shape[1]} columnas")
print(f"  agebs_censo_pct_zecditvallejo.geojson     → {agebs.shape[0]:,} registros | {agebs.shape[1]} columnas")

Exportados:
  manzanas_censo_pct_zecditvallejo.geojson  → 5,700 registros | 38 columnas
  agebs_censo_pct_zecditvallejo.geojson     → 222 registros | 68 columnas


In [ ]:
manzanas_pct.to_crs("EPSG:4326").to_file(os.path.join(ruta_finales, "manzanas_censo_pct_zecditvallejo.geojson"))
